# Import

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import json
import os
import re
import time
import openai
!pip install openai==0.28

In [ ]:
# Update with your own save path
save_dir = "/content/drive/My Drive/Master Thesis/CholecT50"

# Functions

In [ ]:
json_path = f"{save_dir}/Predictions/predicted_clips.json"
with open(json_path, "r") as f:
    prediction_data = json.load(f)

In [ ]:
def get_clip_captions(video_folder, prediction_data):

    clip_captions = []
    video_predictions = prediction_data.get(video_folder, [])
    sorted_predictions = sorted(video_predictions, key=lambda x: int(   re.sub(r"\s*\(\d+\)$", "", x.get("start_frame", "0"))   )  )
    for entry in sorted_predictions:
        start_frame = entry.get("start_frame", "")
        if start_frame.isdigit() and int(start_frame) % 32 == 0:
            predicted_caption = entry.get("predicted_caption", "Caption not found")
            clip_captions.append(predicted_caption)

    return clip_captions

In [ ]:
def generate_report (clip_captions, key):

    openai.api_key = key

    input_text = " ".join(clip_captions)

    prompt = f"""
    Generate a concise and textual surgery report from the following sequential clip captions of a video. Each clip describes a phase of the surgery, including the activity, tools used, and duration.
    **Key Instructions:**
    1. The clips form a continuous video. If multiple clips describe the same activity, combine their durations to reflect the total time spent on that activity.
    2. Write the report in a narrative format, explaining each phase step-by-step in a flowing text.
    Clip captions:
    {input_text}
    """

    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are a medical assistant specialized in generating structured surgery reports."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=2000,
        temperature=0.7,
    )

    report = response['choices'][0]['message']['content']
    return report

# Create reports

In [ ]:
# Complete your openAI key
openai.api_key = ""


In [ ]:

report_path = f"{save_dir}/Predictions/predicted_reports.json"

video_folders = sorted(prediction_data.keys())

if os.path.exists(report_path):
    with open(report_path, "r") as f:
        reports = json.load(f)
else:
    reports = {}


for video_folder in video_folders:
    time.sleep(10)
    clip_captions = get_clip_captions(video_folder, prediction_data)
    report = generate_report(clip_captions, openai.api_key)
    reports[video_folder] = report

    print(f"Generated report for: {video_folder}")


with open(report_path, "w") as f:
    json.dump(reports, f, indent=4)




Generated report for: VID01
Generated report for: VID02
Generated report for: VID04
Generated report for: VID05
Generated report for: VID06
Generated report for: VID08
Generated report for: VID10
Generated report for: VID103
Generated report for: VID110
Generated report for: VID111
Generated report for: VID12
Generated report for: VID13
Generated report for: VID14
Generated report for: VID15
Generated report for: VID18
Generated report for: VID22
Generated report for: VID23
Generated report for: VID25
Generated report for: VID26
Generated report for: VID27
Generated report for: VID29
Generated report for: VID31
Generated report for: VID32
Generated report for: VID35
Generated report for: VID36
Generated report for: VID40
Generated report for: VID42
Generated report for: VID43
Generated report for: VID47
Generated report for: VID48
Generated report for: VID49
Generated report for: VID50
Generated report for: VID51
Generated report for: VID52
Generated report for: VID56
Generated report 

# Example

In [ ]:
reports_path = f"{save_dir}/Predictions/predicted__reports.json"
with open(reports_path, "r") as f:
     reports = json.load(f)


video_folder = "VID06"
report =  reports.get(video_folder, "Report not found")
print(f"Report for {video_folder}:\n\n{report}")


Report for VID06:

The surgical procedure initiated with the preparation phase, which lasted for a total of 56 seconds. Initially, the grasper was engaged in retracting the gallbladder while the hook was used to dissect the gallbladder. Following this, the clipper retracted the omentum for 8 seconds.

The next phase was the Calot triangle dissection, which took an extended duration due to the complexity involved. The hook was used to dissect the gallbladder while the grasper helped in retracting the gallbladder. The dissection of cystic artery and cystic duct was carried out multiple times during this phase. The grasper maintained a consistent presence, assisting in retracting the gallbladder throughout.

The procedure moved on to the clipping and cutting phase which lasted for 117 seconds in total. The grasper was utilised to retract the gallbladder while the clipper was engaged in clipping the cystic artery and the cystic duct. The scissors were also used to cut the cystic duct and a